# Chronic Kidney Disease (CKD) Diagnostic Pipeline
**Student Name:** [Enter Name]  
**Student ID:** [Enter ID]

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score, confusion_matrix, accuracy_score

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

## Task 1: Data Pipeline
**Instructions:**
1. Load the `kidney_disease.csv` file.
2. Drop all categorical (string) columns to isolate the numerical lab values.
3. Handle missing values (`NaN`s).
4. Scale the features to prevent data leakage.
5. Perform an 80/20 train/validation split.

In [ ]:
# Write your Data Pipeline code here


## Task 2: The Engineered Failure (Baseline Model)
**Instructions:** Build and train an unregularised Dense network. Plot the training and validation loss curves to visually prove it overfits.

In [ ]:
# Write your Baseline Model code here


## Task 3: The Clinical Solution (Engineered Model)
**Instructions:** Build a second model using regularisation techniques (e.g., L2, Dropout, LeakyReLU). Train it and plot the curves to demonstrate the reduction in overfitting.

In [ ]:
# Write your Engineered Model code here


## Task 4: Evaluation & Mathematical Justification
**Instructions:**
1. Render the confusion matrices for both models.
2. Print the final validation Recall for both models.
3. Answer the justification question below.

In [ ]:
# Write your Evaluation code here


### Mathematical Justification
**Final Parameter Count of Engineered Model:** [Enter Number]

**Explanation:** [Write your one-sentence explanation here detailing why this parameter capacity and regularisation strategy mathematically prevents memorisation on a 400-patient dataset.]